# 205 Agent custom

在agent中，我们是如何实现自己定义的，带着这个问题我们自己定义一个智能体，我们将使用 OpenAI 函数调用来创建此代理。我们将首先在没有内存的情况下创建它，但随后我们将展示如何添加内存。需要内存才能启用对话。

In [1]:
#设置代理
import os

os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'http://127.0.0.1:10809'

## LLM加载

In [2]:
# 首先，让我们加载将用于控制代理的语言模型。
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## 定义工具

In [3]:
# 接下来，让我们定义一些要使用的工具。让我们编写一个非常简单的 Python 函数来计算传入的单词的长度。
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

## 创建提示

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## 将工具绑定到LLM

In [5]:
# 代理如何知道它可以使用哪些工具？在本例中，我们依赖于 OpenAI 函数调用 LLM，它将函数作为单独的参数，并经过专门训练，可以知道何时调用这些函数。
# 要将我们的工具传递给代理，我们只需要将它们格式化为 OpenAI 函数格式并将它们传递给我们的模型。（通过 bind 对函数进行绑定操作，我们确保每次调用模型时都会传入它们。
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

## 创建代理

In [6]:
# 将这些部分放在一起，我们现在可以创建代理。我们将导入最后两个实用程序函数：一个组件用于格式化中间步骤（代理操作、工具输出对）以输入可发送到模型的消息，以及一个用于将输出消息转换为代理操作/代理完成的组件。
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [7]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
agent_executor.invoke({"input": "How many letters in the word educa"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'How many letters in the word educa',
 'output': 'There are 5 letters in the word "educa".'}

In [9]:
# 如果我们将其与基础LLM进行比较
llm.invoke("How many letters in the word educa")

AIMessage(content='There are 6 letters in the word "educa".')

## 添加内存

我们有一个代理！但是，此代理是无状态的 - 它不记得有关先前交互的任何内容。这意味着你不能轻易提出后续问题。让我们通过添加内存来解决这个问题。

为了做到这一点，我们需要做两件事：

- 在提示中为内存变量添加一个位置

- 跟踪聊天记录

In [10]:
# 首先，让我们在提示中添加一个内存位置。为此，我们为带有键 "chat_history" 的消息添加一个占位符。请注意，我们将其放在新用户输入的上方（以遵循对话流）
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [11]:
# 然后，我们可以设置一个列表来跟踪聊天记录
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [12]:
# 然后我们可以把它们放在一起！
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
# 运行时，我们现在需要将输入和输出作为聊天记录进行跟踪
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}